In [1]:
from sklearn.tree import plot_tree
import math as maths
import matplotlib.pyplot as plt
import numpy as np
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
import openml
import folium
import pandas as pd
import matplotlib.dates as mdates
from sklearn.compose import make_column_transformer
import folium
from folium import plugins
from folium.plugins import HeatMap
openml.datasets.list_datasets(output_format="dataframe")
dataset = openml.datasets.get_dataset(43584)
X, y, _, _ = dataset.get_data(dataset_format="dataframe")

/home/oscar/.local/lib/python3.10/site-packages/openml/datasets/functions.py:438: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  warnings.warn(


In [20]:
X.dropna(axis=0, subset="drivingTime", inplace=True)

In [21]:
X['drivingTime'].isnull().values.any()

False

In [26]:
train_feat, test_feat, train_target, test_target = train_test_split(X[['dayPeriod', 'season', 'dayName', 'rain', 'snow', 'temperature', 'startLatitude', 'startLongitude']], X['geoDistance'], test_size=0.3, random_state=1)

In [27]:
ohe = OneHotEncoder()
ohe.fit_transform(train_feat)

<1457270x1361913 sparse matrix of type '<class 'numpy.float64'>'
	with 11658160 stored elements in Compressed Sparse Row format>

In [28]:
model = DecisionTreeRegressor()
pipeline = make_pipeline(
    make_column_transformer((OneHotEncoder(),['dayPeriod']), (OneHotEncoder(), ['season']), (OneHotEncoder(), ['dayName']))
    , model)
pipeline.fit(train_feat, train_target)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('onehotencoder-1',
                                                  OneHotEncoder(),
                                                  ['dayPeriod']),
                                                 ('onehotencoder-2',
                                                  OneHotEncoder(), ['season']),
                                                 ('onehotencoder-3',
                                                  OneHotEncoder(),
                                                  ['dayName'])])),
                ('decisiontreeregressor', DecisionTreeRegressor())])

In [29]:
from joblib import dump, load
dump(pipeline, 'pipeline_dist.joblib')

['pipeline_dist.joblib']